In [1]:
import os
os.getcwd()

'C:\\Users\\cscpd\\madly_safe\\notebooks'

In [2]:
import pandas as pd

# Ruta relativa desde notebooks/ hacia data/
ruta_excel = "../data/2025_Accidentalidad.xlsx"

df = pd.read_excel(ruta_excel)

df.shape, df.columns


((32327, 19),
 Index(['num_expediente', 'fecha', 'hora', 'localizacion', 'numero',
        'cod_distrito', 'distrito', 'tipo_accidente', 'estado_meteorológico',
        'tipo_vehiculo', 'tipo_persona', 'rango_edad', 'sexo', 'cod_lesividad',
        'lesividad', 'coordenada_x_utm', 'coordenada_y_utm', 'positiva_alcohol',
        'positiva_droga'],
       dtype='object'))

In [3]:
df.head()


,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2025S000056,2025-01-01,00:49:00,CALL. LOPEZ DE HOYOS / CALL. ROS DE OLANO,140,5,CHAMARTÍN,Colisión fronto-lateral,Despejado,Ciclomotor,Conductor,De 30 a 34 años,Hombre,7.0,Asistencia sanitaria sólo en el lugar del acci...,442965.751,4477385.334,N,NaN
1,2025S000056,2025-01-01,00:49:00,CALL. LOPEZ DE HOYOS / CALL. ROS DE OLANO,140,5,CHAMARTÍN,Colisión fronto-lateral,Despejado,Turismo,Conductor,De 40 a 44 años,Hombre,14.0,Sin asistencia sanitaria,442965.751,4477385.334,N,NaN
2,2025S000056,2025-01-01,00:49:00,CALL. LOPEZ DE HOYOS / CALL. ROS DE OLANO,140,5,CHAMARTÍN,Colisión fronto-lateral,Despejado,Turismo,Pasajero,De 40 a 44 años,Mujer,14.0,Sin asistencia sanitaria,442965.751,4477385.334,N,NaN
3,2025S000057,2025-01-01,02:20:00,GTA. RIO ZANCARA / CALL. ARROYO DE POZUELO,1,9,MONCLOA-ARAVACA,Colisión lateral,Despejado,Turismo,Conductor,De 21 a 24 años,Hombre,NaN,NaN,433911.019,4478290.367,N,NaN
4,2025S000057,2025-01-01,02:20:00,GTA. RIO ZANCARA / CALL. ARROYO DE POZUELO,1,9,MONCLOA-ARAVACA,Colisión lateral,Despejado,Turismo,Conductor,De 45 a 49 años,Hombre,NaN,NaN,433911.019,4478290.367,N,NaN


In [4]:
df.dtypes


num_expediente                  object
fecha                   datetime64[ns]
hora                            object
localizacion                    object
numero                          object
cod_distrito                     int64
distrito                        object
tipo_accidente                  object
estado_meteorológico            object
tipo_vehiculo                   object
tipo_persona                    object
rango_edad                      object
sexo                            object
cod_lesividad                  float64
lesividad                       object
coordenada_x_utm               float64
coordenada_y_utm               float64
positiva_alcohol                object
positiva_droga                 float64
dtype: object

In [5]:
# Asegurarnos de que fecha es datetime
df["fecha"] = pd.to_datetime(df["fecha"])

# Extraer la hora como número (0–23)
df["hora_num"] = pd.to_datetime(df["hora"], format="%H:%M:%S").dt.hour
df[["hora", "hora_num"]].head()


,hora,hora_num
0,00:49:00,0
1,00:49:00,0
2,00:49:00,0
3,02:20:00,2
4,02:20:00,2


In [6]:
# Lunes=0, Domingo=6
df["dia_semana_num"] = df["fecha"].dt.dayofweek

# Nombre del día (en español a mano)
map_dia = {
    0: "Lunes",
    1: "Martes",
    2: "Miércoles",
    3: "Jueves",
    4: "Viernes",
    5: "Sábado",
    6: "Domingo",
}
df["dia_semana"] = df["dia_semana_num"].map(map_dia)

# Fin de semana: sábado o domingo
df["es_fin_semana"] = df["dia_semana"].isin(["Sábado", "Domingo"])
df[["fecha", "dia_semana", "es_fin_semana"]].head()


,fecha,dia_semana,es_fin_semana
0,2025-01-01,Miércoles,False
1,2025-01-01,Miércoles,False
2,2025-01-01,Miércoles,False
3,2025-01-01,Miércoles,False
4,2025-01-01,Miércoles,False


In [7]:
def asignar_franja(hora):
    if 0 <= hora <= 5:
        return "Noche_madrugada"
    elif 6 <= hora <= 9:
        return "Manana_punta"
    elif 10 <= hora <= 13:
        return "Manana_media"
    elif 14 <= hora <= 17:
        return "Tarde"
    elif 18 <= hora <= 21:
        return "Tarde_punta"
    elif 22 <= hora <= 23:
        return "Noche"
    else:
        return "Desconocida"

df["franja_horaria"] = df["hora_num"].apply(asignar_franja)
df[["hora", "hora_num", "franja_horaria"]].head()


,hora,hora_num,franja_horaria
0,00:49:00,0,Noche_madrugada
1,00:49:00,0,Noche_madrugada
2,00:49:00,0,Noche_madrugada
3,02:20:00,2,Noche_madrugada
4,02:20:00,2,Noche_madrugada


In [8]:
# Grave = 1 si cod_lesividad en {3,4} (ingreso >24h o fallecido)
# No grave = 0 si cod_lesividad en {1,2,5,6,7,14}
# Dejamos fuera las filas con cod_lesividad NaN

import numpy as np

df["grave"] = np.nan

cond_grave = df["cod_lesividad"].isin([3, 4])
cond_no_grave = df["cod_lesividad"].isin([1, 2, 5, 6, 7, 14])

df.loc[cond_grave, "grave"] = 1
df.loc[cond_no_grave, "grave"] = 0

df["grave"].value_counts(dropna=False)


grave
0.0    17832
NaN    14191
1.0      304
Name: count, dtype: int64

In [9]:
df_target = df.dropna(subset=["grave"]).copy()
df_target["grave"].value_counts(normalize=True)


grave
0.0    0.983238
1.0    0.016762
Name: proportion, dtype: float64

In [12]:
import sys
if ".." not in sys.path:
    sys.path.append("..")

from src.etl import cargar_y_preparar_2025

df_proc, df_target = cargar_y_preparar_2025()

df_proc.shape, df_target.shape



((32327, 25), (18136, 25))

In [13]:
df_proc[["fecha", "hora", "hora_num", "dia_semana", "es_fin_semana", "franja_horaria"]].head()


,fecha,hora,hora_num,dia_semana,es_fin_semana,franja_horaria
0,2025-01-01,00:49:00,0,Miércoles,False,Noche_madrugada
1,2025-01-01,00:49:00,0,Miércoles,False,Noche_madrugada
2,2025-01-01,00:49:00,0,Miércoles,False,Noche_madrugada
3,2025-01-01,02:20:00,2,Miércoles,False,Noche_madrugada
4,2025-01-01,02:20:00,2,Miércoles,False,Noche_madrugada


In [14]:
df_target["grave"].value_counts(normalize=True)


grave
0.0    0.983238
1.0    0.016762
Name: proportion, dtype: float64